In [2]:
# r script that grabs job titles, firm names, and locations from indeed.com
require(rvest)

In [3]:
getinfo <- function(page, tags) {
  page %>%
  html_nodes(tags) %>%   # Select nodes from html document
  html_text()  # Extract attributes, text and tag name from html
}

In [4]:
mylist <- list()

In [5]:
what <- 'data+science'
where <- 'toronto'

In [6]:
url <- "http://www.indeed.ca/jobs?"
q <- "q="
l <- "&l="
url <- paste(url,q,what,l,where,sep="")

In [7]:
url

[1] "http://www.indeed.ca/jobs?q=data+science&l=toronto"

In [8]:
request <- read_html(url)

In [10]:
count <- request %>%
  html_nodes("#searchCount") %>%   ##css selector; use $('#searchCount') in console to get the data
  html_text()

In [12]:
count <- strsplit(count, " ")[[1]][6]

In [13]:
count <- as.numeric(gsub(",","",count))

In [14]:
searchlimit <- if(count<1001) count else 1001

In [29]:
i=0
while (i<searchlimit) { 
  url = "http://www.indeed.ca/jobs?q=data+science&l=toronto"
  start = "&start="
  inc = as.character(i)
  newurl = gsub(" ","",paste(url, start, inc))
  request <- read_html(newurl)
  firms <- getinfo(request, ".company") %>% trimws
  jobs <- getinfo(request, "#resultsCol .jobtitle") %>% trimws
  location <- getinfo(request, ".location")
  location <- gsub("[0-9]","",location) %>% trimws
  summary <- getinfo(request, ".summary") %>% trimws
  mylist$FIRMS <- firms
  mylist$JOBS <- jobs
  mylist$LOCATION <- location
  #mylist$SUMMARY <- summary
  try({
    write.table(mylist, file="./data/my.csv", row.names=FALSE, sep=",", append=TRUE, col.names=FALSE)
  })
  i <- i+10
}